In [2]:
import numpy as np


In [3]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        # Initialize weights with random values
        self.weights1 = np.random.randn(input_size, hidden_size)
        self.weights2 = np.random.randn(hidden_size, output_size)

    def forward(self, X):
        # Forward propagation
        self.z1 = np.dot(X, self.weights1)
        self.a1 = self.sigmoid(self.z1)
        self.z2 = np.dot(self.a1, self.weights2)
        self.y_hat = self.sigmoid(self.z2)
        return self.y_hat

    def backward(self, X, y, y_hat, learning_rate):
        # Backpropagation
        delta2 = (y_hat - y) * self.sigmoid_derivative(self.z2)
        delta1 = delta2.dot(self.weights2.T) * self.sigmoid_derivative(self.z1)
        self.weights2 -= learning_rate * self.a1.T.dot(delta2)
        self.weights1 -= learning_rate * X.T.dot(delta1)

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def sigmoid_derivative(self, z):
        return self.sigmoid(z) * (1 - self.sigmoid(z))


In [14]:
import pandas as pd

# URL where the dataset is hosted
url = "https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv"

# Load dataset
boston = pd.read_csv(url)

# Separate features and target variable
X = boston.drop(columns=["medv"]).values
y = boston["medv"].values

# Example train-test split and normalization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [16]:
# Define network parameters
input_size = X_train.shape[1]  # Number of features
hidden_size = 10                # Number of neurons in the hidden layer
output_size = 1                 # Output size (1 for regression)
learning_rate = 0.01            # Learning rate
num_epochs = 1000               # Number of epochs

# Initialize the neural network
nn = NeuralNetwork(input_size, hidden_size, output_size)

# Train the network
for epoch in range(num_epochs):
    epoch_loss = 0
    for i in range(len(X_train)):
        X = X_train[i].reshape(1, -1)
        y = y_train[i]
        y_hat = nn.forward(X)
        nn.backward(X, y, y_hat, learning_rate)
        epoch_loss += (y_hat - y) ** 2  # Accumulate loss for monitoring

    # Print loss every 100 epochs
    if (epoch + 1) % 100 == 0:
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss/len(X_train)}')


Epoch 100/1000, Loss: [[561.96467379]]
Epoch 200/1000, Loss: [[561.96347994]]
Epoch 300/1000, Loss: [[561.96308863]]
Epoch 400/1000, Loss: [[561.96289475]]
Epoch 500/1000, Loss: [[561.96277912]]
Epoch 600/1000, Loss: [[561.9627024]]
Epoch 700/1000, Loss: [[561.96264779]]
Epoch 800/1000, Loss: [[561.96260696]]
Epoch 900/1000, Loss: [[561.96257528]]
Epoch 1000/1000, Loss: [[561.96255]]


In [17]:
from sklearn.metrics import mean_squared_error

# Make predictions on the test set
predictions = np.array([nn.forward(x.reshape(1, -1)) for x in X_test]).flatten()

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error on Test Set:", mse)


Mean Squared Error on Test Set: 493.1017468866691
